In [ ]:
import sys
import pandas as pd
import pymc3 as pm
import theano.tensor as tt
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

Error: Error: Activating Python 3.6.9 64-bit ('pp': conda) to run Jupyter failed with Error: StdErr from ShellExec, The process cannot access the file because it is being used by another process.
The process cannot access the file because it is being used by another process.
The system cannot find the file C:\Users\DELL\AppData\Local\Temp\conda-4346-27937.tmp.
The process cannot access the file because it is being used by another process.
.

In [0]:
data = pd.read_csv("./EUR_USD_data.csv")
max=data['High'].max(axis=0)
min=data['Low'].min(axis=0)
train = data[['Open','Days']]
trainarray = train.to_numpy()
trainlist = train.values.tolist()
train.describe()


In [0]:
print(trainarray)

In [74]:
T= pd.read_csv("./EUR_USD_test.csv")
# T.head()
test = T[['Open','High','Low']]
# test.head()
print(len(trainarray))

1328


In [0]:
# set the seed
np.random.seed(1)

n = len(trainarray) # The number of data points
X = trainarray # The inputs to the GP, they must be arranged as a column vector

# Define the true covariance function and its parameters
ℓ_true = ([min,max])
η_true = 3.0
cov_func = η_true**2 * pm.gp.cov.Matern52(2,ℓ_true)

# A mean function that is zero everywhere
mean_func = pm.gp.mean.Constant(1.133831)

# The latent function values are one sample from a multivariate normal
# Note that we have to call `eval()` because PyMC3 built on top of Theano
f_true = np.random.multivariate_normal(mean_func(X).eval(), cov_func(X).eval() + 1e-8*np.eye(n), 1).flatten()

# The observed data is the latent function plus a small amount of T distributed noise
# The standard deviation of the noise is `sigma`, and the degrees of freedom is `nu`
σ_true = 2.0
ν_true = 3.0
y = f_true + σ_true * np.random.standard_t(ν_true, size=n)

## Plot the data and the unobserved latent function
fig = plt.figure(figsize=(12,5)); ax = fig.gca()
ax.plot(X, f_true, "dodgerblue", lw=3, label="True f");
ax.plot(X, y, 'ok', ms=3, label="Data");
ax.set_xlabel("X"); ax.set_ylabel("y"); plt.legend();

In [80]:
with pm.Model() as model:
    ℓ = pm.Gamma("ℓ", alpha=2, beta=1)
    η = pm.HalfCauchy("η", beta=5)

    cov = η**2 * pm.gp.cov.Matern52(1, ℓ)
    gp = pm.gp.Latent(cov_func=cov)

    f = gp.prior("f", X=X)

    σ = pm.HalfCauchy("σ", beta=5)
    ν = pm.Gamma("ν", alpha=2, beta=0.1)
    y_ = pm.StudentT("y", mu=f, lam=1.0/σ, nu=ν, observed=y)

    trace = pm.sample(1000, chains=1)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
C:\Users\DELL\Anaconda3\envs\pp\lib\site-packages\theano\tensor\basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
Sequential sampling (1 chains in 1 job)
NUTS: [ν, σ, f_rotated_, η, ℓ]
Sampling chain 0, 0 divergences:   0%|          | 1/1500 [34:02<850:32:09, 2042.65s/it]


SystemError: <class 'range'> returned a result with an error set
Apply node that caused the error: Elemwise{Composite{(((i0 + (i1 * Composite{sqrt((i0 + clip((i1 + i2 + i3), i4, i5)))}(i2, i3, i4, i5, i6, i7)) + (i8 * sqr(Composite{sqrt((i0 + clip((i1 + i2 + i3), i4, i5)))}(i2, i3, i4, i5, i6, i7)))) * exp((i9 * Composite{sqrt((i0 + clip((i1 + i2 + i3), i4, i5)))}(i2, i3, i4, i5, i6, i7))) * i10) + i11)}}[(0, 3)](TensorConstant{(1, 1) of 1.0}, TensorConstant{(1, 1) of ..6797749979}, TensorConstant{(1, 1) of 1e-12}, Dot22Scalar.0, InplaceDimShuffle{0,x}.0, InplaceDimShuffle{x,0}.0, TensorConstant{(1, 1) of 0.0}, TensorConstant{(1, 1) of inf}, TensorConstant{(1, 1) of ..6666666667}, TensorConstant{(1, 1) of ..6797749979}, Elemwise{sqr,no_inplace}.0, TensorConstant{[[1.e-06 0..0 1.e-06]]})
Toposort index: 20
Inputs types: [TensorType(float64, (True, True)), TensorType(float64, (True, True)), TensorType(float64, (True, True)), TensorType(float64, matrix), TensorType(float64, col), TensorTy

In [0]:
lines = [
    ("η",  {}, η_true),
    ("σ", {}, σ_true),
    ("ℓ", {}, ℓ_true),
    ("ν", {}, ν_true),
]
pm.traceplot(trace, lines=lines, var_names=["η", "σ", "ℓ", "ν"]);

In [52]:
# plot the results
fig = plt.figure(figsize=(12,5)); ax = fig.gca()

# plot the samples from the gp posterior with samples and shading
from pymc3.gp.util import plot_gp_dist
plot_gp_dist(ax, trace["f"], X);

# plot the data and the true latent function
plt.plot(X, f_true, "dodgerblue", lw=3, label="True f");
plt.plot(X, y, 'ok', ms=3, alpha=0.5, label="Observed data");

# axis labels and title
plt.xlabel("X"); plt.ylabel("True f(x)");
plt.title("Posterior distribution over $f(x)$ at the observed values"); plt.legend();

25
